In [4]:
import sys
sys.path.append("C:/code/qdc2")
import numpy as np, matplotlib.pyplot as plt
from qdc.mmf.many_wl_fiber import ManyWavelengthFiber
from qdc.mmf.qdc_experiment import QDCMMFExperiment, propagate_free_space

def make_exp(M, is_step_index=False):
    mwf = ManyWavelengthFiber(N_wl=3, npoints=2**8, is_step_index=is_step_index)
    exp = QDCMMFExperiment(mwf)
    exp.set_phase_matching(Lc_um=4000, pump_waist_crystal=500,
                           magnification=M, wl_pump=0.405, n_pump=1.692)
    return mwf, exp

def run_and_collect(M, is_step_index=False):
    mwf, exp = make_exp(M, is_step_index)
    mid = len(mwf.fibers) // 2
    f = mwf.fibers[mid]
    f.set_input_gaussian(*exp.g_params)

    E0 = f.propagate(show=False)
    E_filt = exp._apply_phase_matching(E0)

    n = exp.n
    G = exp._pm_pump_amp if exp._pm_pump_amp is not None else np.ones((n, n))
    S = np.fft.fftshift(exp._pm_filter)

    F0 = np.abs(np.fft.fftshift(np.fft.fft2(E0.reshape(n, n))))
    F1 = np.abs(np.fft.fftshift(np.fft.fft2(E_filt.reshape(n, n))))
    I0 = np.abs(E0.reshape(n, n)) ** 2
    I1 = np.abs(E_filt.reshape(n, n)) ** 2
    return dict(G=G, S=S, I0=I0, I1=I1, F0=F0, F1=F1, dx=mwf.dx)

# -------- plot helper with colorbar --------
def show(ax, img, ttl, dx=None, fiber_radius_um=None):
    extent = None
    if dx is not None:
        n = img.shape[0]
        extent = [-(n // 2) * dx, (n // 2) * dx,
                  -(n // 2) * dx, (n // 2) * dx]

    im = ax.imshow(img, extent=extent)
    ax.set_title(ttl, fontsize=8)
    # ax.axis('off')
    plt.colorbar(im, ax=ax, fraction=0.045, pad=0.02)

    # Add fiber core circle in real space
    if fiber_radius_um is not None and dx is not None:
        from matplotlib.patches import Circle
        circle = Circle((0, 0), radius=fiber_radius_um, edgecolor='red',
                        facecolor='none', lw=1)
        ax.add_patch(circle)

Ms = [3.3, 10]
is_step_index = True
data1 = run_and_collect(M=Ms[0], is_step_index=is_step_index)
data2 = run_and_collect(M=Ms[1], is_step_index=is_step_index)

titles = ['Gaussian', 'sinc PM', 'real |E|² before', 'real |E|² after',
          '|FFT| before', '|FFT| after']
rows = ['G', 'S', 'I0', 'I1', 'F0', 'F1']

fig, axes = plt.subplots(len(rows), 2, figsize=(9, 14))
for col, data, M in zip([0, 1], [data1, data2], Ms):
    dx = data['dx']
    for r, key in enumerate(rows):
        is_real_space = key in ['G', 'I0', 'I1']
        show(axes[r, col], data[key], f'{titles[r]} (M={M})',
             dx=dx if is_real_space else None,
             fiber_radius_um=25 if is_real_space else None)

plt.tight_layout()
plt.show()


Getting 3 fibers...


3it [00:01,  2.27it/s]


Got fibers!
Getting 3 fibers...


3it [00:01,  2.22it/s]


Got fibers!
